# 6.8610  PROJECT

## Install and import libraries

In [1]:
# !pip install nltk
# !pip install datasets
# !pip install transformers[torch]
# !pip install tokenizers
# !pip install evaluate
# !pip install rouge_score
# !pip install sentencepiece
# !pip install huggingface_hub
# !pip install wandb

In [2]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from datasets import Dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import os
import json
import torch
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import gc
import pandas as pd
import random
random.seed(42)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


## Load model

In [4]:
MODEL_NAME = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

C:\Users\samch\micromambaenv\envs\cs109a\lib\site-packages\transformers\models\t5\tokenization_t5.py:238: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If 

## Create datasets

### Create code dataset

In [5]:
def extract_text_before_example(text):
    return text.split("**Example 1:**")[0].strip()

In [6]:
def convert_folder_to_datasetdict_code(folder_path):
    with open(folder_path, 'r') as file:
        data = json.load(file)

    df = pd.json_normalize(data)
    dataset = Dataset.from_pandas(df[['code_with_problem', 'code_only']])
    dataset['question'] = dataset['code_with_problem'].apply(extract_text_before_example)
    return dataset

In [7]:
def convert_folder_to_datasetdict_code(folder_path):
    with open(folder_path, 'r') as file:
        data = json.load(file)

    df = pd.json_normalize(data)

    # Extract text before "Example 1" for the 'code_with_problem' column
    df['question'] = df['code_with_problem'].apply(extract_text_before_example)

    # Rename columns and create the dataset
    df.rename(columns={'code_only': 'answer'}, inplace=True)
    dataset = Dataset.from_pandas(df[['question', 'answer']])

    return dataset

In [8]:
code_dataset = convert_folder_to_datasetdict_code('data/code/leetcode-solutions.json')
train_dataset, test_dataset = train_test_split(code_dataset, test_size=0.2)
train_dataset = Dataset.from_dict(train_dataset)
test_dataset = Dataset.from_dict(test_dataset)

In [9]:
code_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

print(code_dict)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1887
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 472
    })
})


In [10]:
n_train = code_dict['train'].num_rows
n_test = code_dict['test'].num_rows
n_total = n_train + n_test

In [11]:
code_dict["train"][0]

{'question': "# You are given two linked lists: `list1` and `list2` of sizes `n` and `m` respectively.\n\nRemove `list1`'s nodes from the `ath` node to the `bth` node, and put `list2` in their place.\n\nThe blue edges and nodes in the following figure indicate the result:\n\n_Build the result list and return its head._",
 'answer': "```python\ndef minCost(n, cuts):\n    cuts = [0] + cuts + [n]\n    cuts.sort()\n    size = len(cuts)\n    dp = [[0] * size for _ in range(size)]\n\n    for len in range(2, size):\n        for i in range(size - len):\n            j = i + len\n            dp[i][j] = float('inf')\n            for k in range(i + 1, j):\n                dp[i][j] = min(dp[i][j], dp[i][k] + dp[k][j] + cuts[j] - cuts[i])\n\n    return dp[0][size - 1]\n```\n\n"}

### Create general knowledge dataset

In [12]:
csv_file_path = 'data/general/general.csv'
general = pd.read_csv(csv_file_path)
general = general.sample(n=n_total, random_state=42)
general['id'] = range(len(general))
train_sample, test_sample = train_test_split(general, test_size=0.2, random_state=42)
train_sample.reset_index(drop=True, inplace=True)
test_sample.reset_index(drop=True, inplace=True)

train_dataset = Dataset.from_pandas(train_sample[['question', 'answer']])
test_dataset = Dataset.from_pandas(test_sample[['question', 'answer']])

general_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
})

print(general_dict)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1887
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 472
    })
})


In [13]:
general_dict["train"][0]

{'question': 'Add 1 letter to "Iowa" to get the name of this tribe who lived south of the Iowa',
 'answer': 'Kiowa'}

### Create math dataset

In [14]:
def convert_folder_to_datasetdict_math(folder_path):
    #data = {"id": [], "question": [], "level": [], "type": [], "answer": []}
    data = {"question": [], "answer": []}

    subject_dictionary = {
        "algebra": 1,
        "counting_and_probability": 2,
        "geometry": 3,
        "intermediate_algebra": 4,
        "number_theory": 5,
        "prealgebra": 6,
        "precalculus": 7
    }

    for subdir, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(subdir, file)
            folder_name = os.path.basename(os.path.normpath(subdir))
            
            with open(file_path, "r") as f:
                json_data = json.load(f)
                problem = json_data.get("problem", "")
                level = json_data.get("level", "")
                type_ = json_data.get("type", "")
                solution = json_data.get("solution", "")

                # Generate id from subject code and file name
                file_name = os.path.splitext(file)[0]
                subject_code = subject_dictionary.get(folder_name, 0)  # Default to 0 if not found
                id_ = f"{subject_code}_{file_name}"

                #data["id"].append(id_)
                data["question"].append(problem)
                #data["level"].append(level)
                #data["type"].append(type_)
                data["answer"].append(solution)

    dataset = Dataset.from_dict(data)
    return dataset

In [16]:
math_train = convert_folder_to_datasetdict_math("data/math/train/")
math_test = convert_folder_to_datasetdict_math("data/math/test/")

In [17]:
math_train = math_train.shuffle(seed=42)
math_train = math_train.select(range(n_train))

math_test = math_test.shuffle(seed=42)
math_test = math_test.select(range(n_test))

math_dict = DatasetDict({
    'train': math_train,
    'test': math_test
})

print(math_dict)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1887
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 472
    })
})


In [18]:
math_dict["train"][0]

{'question': 'What is the number of units in the distance between $(2,5)$ and $(-6,-1)$?',
 'answer': 'We use the distance formula: $\\sqrt{(-6 - 2)^2 + (-1 - 5)^2},$ so then we find that $\\sqrt{64 + 36} = \\boxed{10}$.\n\n- OR -\n\nWe note that the points $(2, 5)$, $(-6, -1)$, and $(2, -1)$ form a right triangle with legs of length 6 and 8. This is a Pythagorean triple, so the length of the hypotenuse must be $\\boxed{10}$.'}

### Create 50% samples

In [21]:
math_train_sample = math_dict['train'].shuffle(seed=42).select([i for i in range(math_dict['train'].num_rows // 2)])
math_test_sample = math_dict['test'].shuffle(seed=42).select([i for i in range(math_dict['test'].num_rows // 2)])

general_train_sample = general_dict['train'].shuffle(seed=42).select([i for i in range(general_dict['train'].num_rows // 2+1)])
general_test_sample = general_dict['test'].shuffle(seed=42).select([i for i in range(general_dict['test'].num_rows // 2)])

code_train_sample = code_dict['train'].shuffle(seed=42).select([i for i in range(code_dict['train'].num_rows // 2)])
code_test_sample = code_dict['test'].shuffle(seed=42).select([i for i in range(code_dict['test'].num_rows // 2)])

### Create 50% general 50% math dataset


In [22]:
general_math_dict = DatasetDict({
    'train': Dataset.from_dict({
        'question': math_train_sample['question'] + general_train_sample['question'],
        'answer': math_train_sample['answer'] + general_train_sample['answer'],
    }),
    'test': Dataset.from_dict({
        'question': math_test_sample['question'] + general_test_sample['question'],
        'answer': math_test_sample['answer'] + general_test_sample['answer'],
    })
})


general_math_dict = DatasetDict({
    'train': general_math_dict['train'].shuffle(seed=42),
    'test': general_math_dict['test'].shuffle(seed=42)
})

In [23]:
general_math_dict["train"][0]

{'question': 'Find the monic quadratic polynomial, in $x,$ with real coefficients, which has $1 - i$ as a root.',
 'answer': 'If a polynomial has real coefficients, then any complex conjugate of a root must also be a root.  Hence, the other root is $1 + i.$  Thus, the polynomial is\n\\[(x - 1 - i)(x - 1 + i) = (x - 1)^2 - i^2 = \\boxed{x^2 - 2x + 2}.\\]'}

### Create 50% general 50% code dataset


In [24]:
general_code_dict = DatasetDict({
    'train': Dataset.from_dict({
        'question': code_train_sample['question'] + general_train_sample['question'],
        'answer': code_train_sample['answer'] + general_train_sample['answer'],
    }),
    'test': Dataset.from_dict({
        'question': code_test_sample['question'] + general_test_sample['question'],
        'answer': code_test_sample['answer'] + general_test_sample['answer'],
    })
})

general_code_dict = DatasetDict({
    'train': general_code_dict['train'].shuffle(seed=42),
    'test': general_code_dict['test'].shuffle(seed=42)
})

print(general_code_dict)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1887
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 472
    })
})


In [25]:
general_code_dict["train"][0]

{'question': '# Given a string `s`, return _the last substring of_ `s` _in lexicographical order_.',
 'answer': '```python\ndef lastSubstring(s: str) -> str:\n    maxIndex = 0\n    curIndex = 1\n    while curIndex < len(s):\n        i = 0\n        while curIndex + i < len(s) and s[maxIndex + i] == s[curIndex + i]:\n            i += 1\n        if curIndex + i == len(s):\n            break\n        if s[maxIndex + i] < s[curIndex + i]:\n            maxIndex = curIndex\n        curIndex += 1\n    return s[maxIndex:]\n```\n\n'}

## Preprocessing

In [26]:
prefix = "Please answer this question: "

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["question"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)
  
   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["answer"], 
                      max_length=512,         
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [27]:
tokenized_dataset_math = math_dict.map(preprocess_function, batched=True)
tokenized_dataset_code = code_dict.map(preprocess_function, batched=True)
tokenized_dataset_general = general_dict.map(preprocess_function, batched=True)
tokenized_dataset_general_code = general_code_dict.map(preprocess_function, batched=True)
tokenized_dataset_general_math = general_math_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/1887 [00:00<?, ? examples/s]

Map:   0%|          | 0/472 [00:00<?, ? examples/s]

Map:   0%|          | 0/1887 [00:00<?, ? examples/s]

Map:   0%|          | 0/472 [00:00<?, ? examples/s]

Map:   0%|          | 0/1887 [00:00<?, ? examples/s]

Map:   0%|          | 0/472 [00:00<?, ? examples/s]

Map:   0%|          | 0/1887 [00:00<?, ? examples/s]

Map:   0%|          | 0/472 [00:00<?, ? examples/s]

Map:   0%|          | 0/1887 [00:00<?, ? examples/s]

Map:   0%|          | 0/472 [00:00<?, ? examples/s]

In [28]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

In [29]:
def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  
   return result

## Fine-tuning

### Experiment

In [46]:
def training_args(output_dir, L_RATE = 3e-4, BATCH_SIZE = 4, PER_DEVICE_EVAL_BATCH = 4, WEIGHT_DECAY = 0.01, SAVE_TOTAL_LIM = 3, NUM_EPOCHS = 3, OVERWRITE_OUTPUT_DIR = True, LOAD_BEST_MODEL_AT_END = True):
    training_args = Seq2SeqTrainingArguments(
       output_dir=output_dir,
       overwrite_output_dir=OVERWRITE_OUTPUT_DIR,
       save_strategy="epoch",
       evaluation_strategy="epoch",
       load_best_model_at_end=LOAD_BEST_MODEL_AT_END,
       learning_rate=L_RATE,
       per_device_train_batch_size=BATCH_SIZE,
       per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
       weight_decay=WEIGHT_DECAY,
       save_total_limit=SAVE_TOTAL_LIM,
       num_train_epochs=NUM_EPOCHS,
       predict_with_generate=True,
       push_to_hub=False
    )
    return training_args

In [47]:
lrs = [3e-3, 3e-4, 3e-5] 

### Math

In [48]:
for lr in lrs:
    model_math = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
    data_collator_math = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_math)
    
    output_dir_root = "./results/math"
    output_dir = f"{output_dir_root}/{lr:.0e}".replace("0", "")
    training_args_math = training_args(L_RATE = lr, BATCH_SIZE = 4, NUM_EPOCHS = 3, output_dir=output_dir)

    trainer_math = Seq2SeqTrainer(
       model=model_math,
       args=training_args_math,
       train_dataset=tokenized_dataset_math["train"], 
       eval_dataset=tokenized_dataset_math["test"],   
       tokenizer=tokenizer,
       data_collator=data_collator_math,
       compute_metrics=compute_metrics
    )

    trainer_math.train()

    del model_math
    gc.collect()
    torch.cuda.empty_cache()

### Code

In [49]:
for lr in lrs:
    model_code = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
    data_collator_code = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_code)
    
    output_dir_root = "./results/code"
    output_dir = f"{output_dir_root}/{lr:.0e}".replace("0", "")
    training_args_code = training_args(L_RATE = lr, BATCH_SIZE = 4, NUM_EPOCHS = 3, output_dir=output_dir)

    trainer_code = Seq2SeqTrainer(
       model=model_code,
       args=training_args_code,
       train_dataset=tokenized_dataset_code["train"], 
       eval_dataset=tokenized_dataset_code["test"],   
       tokenizer=tokenizer,
       data_collator=data_collator_code,
       compute_metrics=compute_metrics
    )

    trainer_code.train()

    del model_code
    gc.collect()
    torch.cuda.empty_cache()

### General

In [50]:
for lr in lrs:
    model_general = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
    data_collator_general = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_general)

    output_dir_root = "./results/general"
    output_dir = f"{output_dir_root}/{lr:.0e}".replace("0", "")
    training_args_general = training_args(L_RATE = lr, BATCH_SIZE = 4, NUM_EPOCHS = 3, output_dir=output_dir)

    trainer_general = Seq2SeqTrainer(
       model=model_general,
       args=training_args_general,
       train_dataset=tokenized_dataset_general["train"], 
       eval_dataset=tokenized_dataset_general["test"],   
       tokenizer=tokenizer,
       data_collator=data_collator_general,
       compute_metrics=compute_metrics
    )

    trainer_general.train()

    del model_general
    gc.collect()
    torch.cuda.empty_cache()

### General + Code

In [51]:
for lr in lrs:
    model_general_code = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
    data_collator_general_code = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_general_code)

    output_dir_root = "./results/general_code"
    output_dir = f"{output_dir_root}/{lr:.0e}".replace("0", "")
    training_args_general_code = training_args(L_RATE = lr, BATCH_SIZE = 4, NUM_EPOCHS = 3, output_dir=output_dir)

    trainer_general_code = Seq2SeqTrainer(
       model=model_general_code,
       args=training_args_general_code,
       train_dataset=tokenized_dataset_general_code["train"], 
       eval_dataset=tokenized_dataset_general_code["test"],   
       tokenizer=tokenizer,
       data_collator=data_collator_general_code,
       compute_metrics=compute_metrics
    )

    trainer_general_code.train()

    del model_general_code
    gc.collect()
    torch.cuda.empty_cache()

### General + Math

In [52]:
for lr in lrs:
    model_general_math = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
    data_collator_general_math = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_general_math)

    output_dir_root = "./results/general_math"
    output_dir = f"{output_dir_root}/{lr:.0e}".replace("0", "")
    training_args_general_math = training_args(L_RATE = lr, BATCH_SIZE = 4, NUM_EPOCHS = 3, output_dir=output_dir)

    trainer_general_math = Seq2SeqTrainer(
       model=model_general_math,
       args=training_args_general_math,
       train_dataset=tokenized_dataset_general_math["train"], 
       eval_dataset=tokenized_dataset_general_math["test"],   
       tokenizer=tokenizer,
       data_collator=data_collator_general_math,
       compute_metrics=compute_metrics
    )

    trainer_general_math.train()

    del model_general_math
    gc.collect()
    torch.cuda.empty_cache()